In [ ]:
import requests
from openai import OpenAI
from dotenv import load_dotenv, find_dotenv
import datetime

load_dotenv(find_dotenv())

client = OpenAI()

def get_opening_hours():
    try:
        response = requests.get('http://localhost:8000/opening-hours')
        if response.status_code == 200:
            return response.json()
        else:
            return "Error: Unable to fetch opening hours."
    except requests.exceptions.RequestException as e:
        return f"Request failed: {e}"

def how_late():
    return datetime.datetime.now().strftime("%H:%M:%S")

file = client.files.create(
    file=open("knowledge.txt", "rb"),
    purpose='assistants'
)

In [ ]:
assistant = client.beta.assistants.create(
    name="Summariser",
    description="You are great at assisting customers of a bank.",
    model="gpt-4-turbo-preview",
    tools=[
        {"type": "code_interpreter"},
        {"type": "retrieval"},
        {
            "type": "function",
            "function": {
                "name": "get_opening_hours",
                "description": "Get the opening hours of the bank",
                "parameters": {
                    "type": "object",
                    "properties": {},
                    "required": []
                }
            }
        },
        {
            "type": "function",
            "function": {
                "name": "how_late",
                "description": "Get the current time",
                "parameters": {
                    "type": "object",
                    "properties": {},
                    "required": []
                }
            }
        }
    ],
    file_ids=[file.id]
)
print(assistant)

In [ ]:
thread = client.beta.threads.create()

In [ ]:
message = client.beta.threads.messages.create(
    thread_id=thread.id,
    role="user",
    content="How much does a loan cost per year?"
)

In [ ]:
run = client.beta.threads.runs.create(
  thread_id=thread.id,
  assistant_id=assistant.id,
  instructions="Please address the user as Jane Doe. The user has a premium account."
)

In [ ]:
run = client.beta.threads.runs.retrieve(
  thread_id=thread.id,
  run_id=run.id
)
print(run.status)

Polling is required the get the current status

In [ ]:
import time

def get_run_status(client, thread_id, run_id):
    try:
        run = client.beta.threads.runs.retrieve(thread_id=thread_id, run_id=run_id)
        return {"status": run.status, "run": run}
    except Exception as e:
        print(f"Error retrieving run status: {e}")
        return {"status": None, "run": None}

def poll_run_status(client, thread_id, run_id):
    while True:
        result = get_run_status(client, thread_id, run_id)
        status = result["status"]
        run = result["run"]
        if status in ['completed', 'cancelled', 'failed', 'requires_action']:
            print(f"Run has finished or requires action with status: {status}")
            break
        else:
            print("Run is not finished yet...")
            time.sleep(2)

    return {"status": status, "run": run}

In [ ]:
result = poll_run_status(client=client, thread_id=thread.id, run_id=run.id)

In [ ]:
print(result["run"])

In [ ]:
messages = client.beta.threads.messages.list(
  thread_id=thread.id
)
print(messages.data)

In [ ]:
for message in messages.data:
    if message.content and message.content[0].text:
        print(f"Message: {message.content[0].text.value}")

### Function usage

In [ ]:
message = client.beta.threads.messages.create(
    thread_id=thread.id, role="user", content="How late is it?"
)

In [ ]:
run = client.beta.threads.runs.create(
  thread_id=thread.id,
  assistant_id=assistant.id,
  instructions="Please address the user as Jane Doe. The user has a premium account.",
)

In [ ]:
run.status

In [ ]:
run = client.beta.threads.runs.retrieve(thread_id=thread.id, run_id=run.id)

In [ ]:
run.required_action.submit_tool_outputs.tool_calls

In [ ]:
for tool_call in run.required_action.submit_tool_outputs.tool_calls:
    tool_call_id = tool_call.id
    function_name = tool_call.function.name
    print(f"Executing function: {function_name}")

    if function_name == "get_opening_hours":
        output = get_opening_hours()
    elif function_name == "how_late":
        output = how_late()
    else:
        output = "Function not recognized"
        print("Error: Function not recognized.")
    print(f"Output from function call:  {output}")

    client.beta.threads.runs.submit_tool_outputs(
    thread_id=thread.id,
    run_id=run.id,
    tool_outputs=[
        {"tool_call_id": tool_call_id, "output": output},
    ])

In [ ]:
print(output)

In [ ]:
run = client.beta.threads.runs.retrieve(thread_id=thread.id, run_id=run.id)

In [ ]:
messages = client.beta.threads.messages.list(
  thread_id=thread.id
)

In [ ]:
for message in messages.data:
    if message.content and message.content[0].text:
        print(f"Message: {message.content[0].text.value}")

### Delete everthing after using it!

In [ ]:
deleted_assistant_file = client.beta.assistants.files.delete(
    assistant_id=assistant.id,
    file_id=file.id
)
print(deleted_assistant_file)
response = client.beta.assistants.delete(assistant.id)
print(response)